In [22]:
import google.generativeai as genai
import google.generativeai as genai
import ast
import os
import random
from nest_asyncio import apply
from templates import templates
import json
from image_agent import get_images
from youtube import get_data
import asyncio
apply()

# Load environment variables from a .env file
from dotenv import load_dotenv

load_dotenv()

from wikipedia_tool import wikipedia_tool

# Parse API keys stored in an environment variable and convert them into a Python list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

# Shuffle the API keys list to ensure usage of different keys over time
random.shuffle(KEY_LIST)

# Initialize a global index to track the current API key being used
current_api_key_index = 0

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))


# List to store historical messages for reference or logging
messages = []


def cycle_api_key():
    """Retrieve the next API key from the list, cycling back to the start if necessary."""
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_new_model():
    """Generate and configure a new AI model instance with a cycled API key."""
    global current_api_key_index
    global messages
    api_key = cycle_api_key()  # Cycle to the next API key

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)

    # Initialize the model with specific configurations
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            temperature=0,  # Set deterministic behavior for the model
            max_output_tokens=8000
        ),
    )
    return model

In [23]:
topic = "Red Black Trees in computer programming"

sources = wikipedia_tool.run(topic)
print(sources)

Page: Red–black tree
Summary: In computer science, a red–black tree is a specialised binary search tree data structure noted for fast storage and retrieval of ordered information, and a guarantee that operations will complete within a known time. Compared to other self-balancing binary search trees, the nodes in a red-black tree hold an extra bit called "color" representing "red" and "black" which is used when re-organising the tree to ensure that it is always approximately balanced.
When the tree is modified, the new tree is rearranged and "repainted" to restore the coloring properties that constrain how unbalanced the tree can become in the worst case. The properties are designed such that this rearranging and recoloring can be performed efficiently.
The (re-)balancing is not perfect, but guarantees searching in Big O time of 
  
    
      
        O
        (
        log
        ⁡
        n
        )
      
    
    {\displaystyle O(\log n)}
  
, where 
  
    
      
        n
   

In [24]:
model = generate_new_model()
audio, video = get_data(topic)

Video downloaded to /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/video.mp4
Audio downloaded to /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/audio.mp3
Extracting /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/video.mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 1 frames
Uploading 1 files. This might take a bit...
Uploading: content/frames/video/vid_frame00_00.jpg...
Completed file uploads!

Uploaded: 1 files


In [25]:
def sources_to_lecture(original_prompt, sources, audio, video):
    """Converts a list of sources into a single lecture template.

    Args:
        sources (list): A list of sources, each containing a 'content' key with the source content.

    Returns:
        str: A single lecture template combining all the source content.
    """
    prompt = (
        "Available templates: \n\n"
        + str(templates)
        + "\n\n"
        + "Original Topic: "
        + original_prompt
        + "\n\n"
        + """
You are an advanced assistant that is in charge of aggregating multiple sources of information into a lecture based on a specific prompt.
Output 8 different slides on the topic given above using the sources provided as well as the given templates.
Make sure the slides flow logically and are easy to understand. Use the correct templates for the content you are presenting. 
Keep text content short and concise, at most 8 words per text.
Make sure you always start off with a title slide.

Try to make each slide have different information.

Return a json parsable string of lectures that are generated from the sources provided.
Make sure the response is in the following format, only output the keys and values shown below:
{
    "title": title of the lecture,
    "description" : description of the lecture,
    "slides": [
        {
          "title": title of the slide,,
          "template_id": 1,
          "texts" : [
              "a stack is a data structure",
              "stacks are used in DFS"
          ],
          "speaker_notes" : A script for the speaker to read
        }   
    ]
}
"""
    )
    lecture = model.generate_content(
        video
        + [
            audio,
            sources + prompt,
        ],
        request_options={"timeout": 1000},
    )
    return lecture.text

result = sources_to_lecture(topic, sources, audio, video)
if "```json" in result:
    # Get the JSON content from the result
    result = result.split("```json")[1]
    result = result.split("```")[0]

result = json.loads(result)
print(result)

{'title': 'Introduction to Red-Black Trees', 'description': 'Exploring the world of balanced search trees and their efficient operations', 'slides': [{'title': 'Red-Black Trees: Intro in 4', 'template_id': 4}, {'title': 'Binary Search Trees', 'template_id': 0, 'texts': ['Ordered binary trees', 'Smaller items to the left', 'Larger items to the right'], 'speaker_notes': 'Binary search trees are fundamental data structures that maintain order among their elements. Each node in the tree has at most two children, with smaller values residing in the left subtree and larger values in the right subtree. This property enables efficient searching, insertion, and deletion operations.'}, {'title': 'Balanced Search Trees', 'template_id': 2, 'texts': ['Guaranteed height of O(log n)', 'Efficient search, insert, and delete', 'Red-black trees are a type of balanced search tree'], 'image': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Red-black_tree_example.svg/512px-Red-black_tree_example.

In [26]:
tasks = []
new_slides = []

for slide in result['slides']:
    template = [t for t in templates if t['template_id'] == slide['template_id']][0]
    num_images = template['num_images']

    if num_images == 0:
        print(num_images)
        new_slides.append({**slide, "images": []})
    else:
        topic = slide['title']
        print(topic)
        print(num_images)
        # Schedule the get_images task for concurrent execution
        task = get_images(topic, num_images)
        tasks.append(task)

# Run all tasks concurrently and collect results
images_results = await asyncio.gather(*tasks)

# Iterate over the slides that require images
image_index = 0
for slide in result['slides']:
    template = [t for t in templates if t['template_id'] == slide['template_id']][0]
    num_images = template['num_images']
    if num_images != 0:
        new_slides.append({
            **slide,
            "images": images_results[image_index]
        })
        image_index += 1
lecture = {
    **result,
    "slides": new_slides
}

0
Binary Search Trees
1
Balanced Search Trees
1
Red-Black Tree Properties
1
Black Height
1
Operations on Red-Black Trees
1
Rotations
1
Space and Time Complexity
1


In [27]:
lecture

{'title': 'Introduction to Red-Black Trees',
 'description': 'Exploring the world of balanced search trees and their efficient operations',
 'slides': [{'title': 'Red-Black Trees: Intro in 4',
   'template_id': 4,
   'images': []},
  {'title': 'Binary Search Trees',
   'template_id': 0,
   'texts': ['Ordered binary trees',
    'Smaller items to the left',
    'Larger items to the right'],
   'speaker_notes': 'Binary search trees are fundamental data structures that maintain order among their elements. Each node in the tree has at most two children, with smaller values residing in the left subtree and larger values in the right subtree. This property enables efficient searching, insertion, and deletion operations.',
   'images': [{'src': 'https://media.geeksforgeeks.org/wp-content/cdn-uploads/20221215114732/bst-21.png',
     'description': 'This image illustrates a Binary Search Tree (BST), a fundamental data structure used in computer science for managing sorted data efficiently. Each 